# First print your name in the cell below then save this file. (or something nice about your instructor)

In [32]:
# In this cell print your name.
print("Victoria Dynak")

Victoria Dynak


# Data wrangling with Pandas exercise
* For this exercise we will be using the `listings.csv` data file.

In [33]:
import pandas as pd
import numpy as np

# Load in the data file using `pd.read_csv()`

In [34]:
# Load data here
df = pd.read_csv("/workspaces/ds-fall-2025-fri-0630/Week-01-Pandas/data/listings.csv")
df.head()

,id,name,host_id,host_name,neighbourhood_group,neighbourhood,latitude,longitude,room_type,price,minimum_nights,number_of_reviews,last_review,reviews_per_month,calculated_host_listings_count,availability_365
0,2595,Skylit Midtown Castle,2845,Jennifer,Manhattan,Midtown,40.75362,-73.98377,Entire home/apt,175,3,48,2019-11-04,0.37,2,365
1,3831,"Whole flr w/private bdrm, bath & kitchen(pls r...",4869,LisaRoxanne,Brooklyn,Clinton Hill,40.68514,-73.95976,Entire home/apt,75,1,340,2020-08-01,4.75,1,265
2,5121,BlissArtsSpace!,7356,Garon,Brooklyn,Bedford-Stuyvesant,40.68688,-73.95596,Private room,60,29,50,2019-12-02,0.37,1,365
3,5136,"Spacious Brooklyn Duplex, Patio + Garden",7378,Rebecca,Brooklyn,Sunset Park,40.66120,-73.99423,Entire home/apt,175,14,1,2014-01-02,0.01,1,295
4,5178,Large Furnished Room Near B'way,8967,Shunichi,Manhattan,Hell's Kitchen,40.76489,-73.98493,Private room,65,2,473,2020-03-15,3.44,1,340


## Exercise 2 - Filtering

Return the following subsets of the dataframe.

1. How many listings are there with a price less than 100? 


2. Find how many listings there are in just Brooklyn.


3. Find how many listings there are in Brooklyn with a price less than 100.


4. Using `.isin()` select anyone that has the host name of Michael, David, John, and Daniel.


5. Create a new column called `adjusted_price` that has $100 added to every listing in Williamsburg.  The prices for all other listings should be the same as the were before. 


6. What % of the rooms are private, and what % of the rooms are shared.  
    * Hint, use `.value_counts()`


In [35]:
# 1. How many listings are there with a price less than 100? 
listings_under_100 = (df["price"] < 100).sum()
print(listings_under_100)

22778


In [36]:
# 2. Make a new DataFrame of listings in Brooklyn named `df_bk` 
# and find how many listings in just Brooklyn.
df_bk = (df["neighbourhood_group"] == "Brooklyn")
brooklyn_count = len(df_bk)
print(brooklyn_count)

46527


In [37]:
# 3. Find how many listings there are in Brooklyn with a price less than 100.
#bk_listings_under_100 = df_bk[df_bk["price"] < 100]
#bk_listings_under_100_count = len(bk_listings_under_100)
bk_listings_under_100 = df[(df["neighbourhood_group"] == "Brooklyn") & (df["price"] < 100)]
bk_listings_under_100_count = len(bk_listings_under_100)
print(bk_listings_under_100_count)

10473


In [38]:
# 4. Using `.isin()` select anyone that has the host name of Michael, David, John, and Daniel.
# How many total are there that have those names
hosts = ["Michael", "David", "John", "Daniel"]
hosts_selected = df[df["host_name"].isin(hosts)]
host_counts = len(hosts_selected)
print(host_counts)

1258


In [39]:
# 5. Create a new column called `adjusted_price` that has $100 added to every listing in Williamsburg.  
# The prices for all other listings should be the same as the were before. 
df["adjusted_price"] = np.where(
    df["neighbourhood"] == "Williamsburg",
    df["price"] + 100, 
    df["price"]
)
print(df[["neighbourhood", "price", "adjusted_price"]].head(40))

                neighbourhood  price  adjusted_price
0                     Midtown    175             175
1                Clinton Hill     75              75
2          Bedford-Stuyvesant     60              60
3                 Sunset Park    175             175
4              Hell's Kitchen     65              65
5             Upper West Side     75              75
6                   Chinatown    140             140
7                West Village    160             160
8                 South Slope     88              88
9             Upper West Side     85              85
10               West Village     65              65
11               Williamsburg    109             209
12                East Harlem    239             239
13                Fort Greene    182             182
14                East Harlem     96              96
15               Williamsburg    299             399
16         Bedford-Stuyvesant    120             120
17            Windsor Terrace    135          

In [40]:
# 6. What % of the rooms are private, and what % of the rooms are shared.  
room_percents = df["room_type"].value_counts(normalize = True) * 100
private_percents = room_percents.get("Private room", 0.0)
shared_percents = room_percents.get("Shared room", 0.0)
print(pd.DataFrame({
    "Room Type": ["Private", "Shared"],
    "Percentage": [private_percents, shared_percents]
}))

  Room Type  Percentage
0   Private   45.444581
1    Shared    2.121349


# Exercise 3 - Grouping

1. Using `groupby`, count how many listings are in each neighbourhood_group.


2. Using `groupby`, find the mean price for each of the neighbourhood_groups. 


3. Using `groupby` and `.agg()`, find the min and max price for each of the neighbourhood_groups. 


4. Using `groupby`, find the median price for each room type in each neighbourhood_group.


5. Using `groupby` and `.agg()`, find the count, min, max, mean, median, and std of the prices for each room type in each neighbourhood_group.

In [41]:
# 1. Using `groupby`, count how many listings are in each neighbourhood_group.
counts_by_borough = df.groupby("neighbourhood_group")["id"].count()
print(counts_by_borough)

neighbourhood_group
Bronx             1183
Brooklyn         18632
Manhattan        20580
Queens            5791
Staten Island      341
Name: id, dtype: int64


In [42]:
# 2. Using `groupby`, find the mean price for each of the neighbourhood_groups. 
mean_price_per_borough = df.groupby("neighbourhood_group")["price"].mean()
print(mean_price_per_borough)

neighbourhood_group
Bronx             92.751479
Brooklyn         120.225258
Manhattan        191.880466
Queens            99.754965
Staten Island    110.947214
Name: price, dtype: float64


In [43]:
# 2.5. Using `groupby`, find the mean price for each room_type. 
mean_room_price = df.groupby("room_type")["price"].mean()
print(mean_room_price)

room_type
Entire home/apt    199.395950
Hotel room         275.015075
Private room        91.453084
Shared room         87.063830
Name: price, dtype: float64


In [44]:
# 3. Using `groupby` and `.agg()`, find the min and max price for each of the neighbourhood_groups. 
mm_borough = df.groupby("neighbourhood_group")["price"].agg(["min", "max"])
print(mm_borough)

                     min    max
neighbourhood_group            
Bronx                 16   1404
Brooklyn               0  10000
Manhattan              0  10000
Queens                 0  10000
Staten Island         19   1200


In [45]:
# 4. Using `groupby`, find the mean price for each room_type in each neighbourhood_group.
mean_room_type_borough = df.groupby(["neighbourhood_group", "room_type"])["price"].mean()
print(mean_room_type_borough)

neighbourhood_group  room_type      
Bronx                Entire home/apt    138.004819
                     Private room        68.419668
                     Shared room         66.391304
Brooklyn             Entire home/apt    171.587687
                     Hotel room         147.300000
                     Private room        71.291189
                     Shared room         57.870091
Manhattan            Entire home/apt    231.335572
                     Hotel room         292.515670
                     Private room       128.277069
                     Shared room        111.735084
Queens               Entire home/apt    150.168900
                     Hotel room         139.058824
                     Private room        69.972564
                     Shared room         89.891892
Staten Island        Entire home/apt    151.720930
                     Private room        70.312883
                     Shared room         46.000000
Name: price, dtype: float64


In [46]:
# 5. Using `groupby` and `.agg()`, find the count, min, max, mean, median, and std of the prices 
# for each room type in each neighbourhood_group.
borough_room_type_stats = (df.groupby(["neighbourhood_group", "room_type"])["price"].agg(count = "count", min = "min", max = "max", mean = "mean", median = "median", std = "std"))
print(borough_room_type_stats)

                                     count  min    max        mean  median  \
neighbourhood_group room_type                                                
Bronx               Entire home/apt    415   25   1404  138.004819   103.0   
                    Private room       722   16    700   68.419668    55.0   
                    Shared room         46   20    800   66.391304    44.0   
Brooklyn            Entire home/apt   9112   20  10000  171.587687   135.0   
                    Hotel room          30    0    399  147.300000   129.0   
                    Private room      9159   10   2500   71.291189    60.0   
                    Shared room        331   15   1500   57.870091    36.0   
Manhattan           Entire home/apt  12209    0  10000  231.335572   165.0   
                    Hotel room         351    0   2211  292.515670   210.0   
                    Private room      7601   10  10000  128.277069    80.0   
                    Shared room        419   10  10000  111.7350

# Load, Merge, and Export files.
1. Load the `prices.csv` and the `n_listings.csv`
    * Having an error..? Inspect the actual csv file if you're having trouble

2. Do join that keeps all the records for each table.
    * Neighbourhood groups should include ['Bronx', 'Brooklyn', 'Manhattan', 'Queens', 'Staten Island',
       'LongIsland']

       
3. Save your joined csv as `joined.csv` into the data folder. 


4. Load your saved table and see if it looks the same or different that the DataFrame you used to create it. 

In [47]:
# 1. Load the `prices.csv` and the `n_listings.csv`
df_prices = pd.read_csv("/workspaces/ds-fall-2025-fri-0630/Week-01-Pandas/data/prices.csv")
df_n_listings = pd.read_csv("/workspaces/ds-fall-2025-fri-0630/Week-01-Pandas/data/n_listings.csv", delimiter = ";")
print(df_prices.head())
print(df_n_listings.head())

  neighbourhood_group   mean_price
0               Bronx    92.751479
1            Brooklyn   120.225258
2           Manhattan   191.880466
3              Queens    99.754965
4       Staten Island   110.947214
  neighbourhood_group  n_listings
0               Bronx        1183
1            Brooklyn       18632
2           Manhattan       20580
3          LongIsland        4121


In [48]:
# 2. Do join that keeps all the records for each table.
print(df_prices.columns.tolist())
print(df_n_listings.columns.tolist())
merged = pd.merge(df_prices, df_n_listings, on = "neighbourhood_group", how = "outer")
print(merged.head())


['neighbourhood_group', ' mean_price']
['neighbourhood_group', 'n_listings']
  neighbourhood_group   mean_price  n_listings
0               Bronx    92.751479      1183.0
1            Brooklyn   120.225258     18632.0
2          LongIsland          NaN      4121.0
3           Manhattan   191.880466     20580.0
4              Queens    99.754965         NaN


In [49]:
# 3. Save your joined csv as `joined.csv` into the data folder.
merged.to_csv("/workspaces/ds-fall-2025-fri-0630/Week-01-Pandas/data/joined.csv", index = False)

In [50]:
# 4. Load your newly saved file, see if it looks the same.  If not, try saving with argument `index=False`
merged_check = pd.read_csv("/workspaces/ds-fall-2025-fri-0630/Week-01-Pandas/data/joined.csv")
print(merged_check.head())

  neighbourhood_group   mean_price  n_listings
0               Bronx    92.751479      1183.0
1            Brooklyn   120.225258     18632.0
2          LongIsland          NaN      4121.0
3           Manhattan   191.880466     20580.0
4              Queens    99.754965         NaN


# Extra Credit
Every question below this cell is extra credit and optional.

### 1. (Easy) Explore this new PandasAI Package and tell me what its all about because I've never used it. 
* https://www.youtube.com/watch?v=5w6eZaoDVVk&ab_channel=CodingIsFun  
* See if you can use it on the listings.csv to find out some cool info. or answer some of the questions above. 

PandasAi uses LLM's to auto gen and run pandas code, so you can perform queries and visualizations with natural langauge.

In [51]:
!pip install pandasai

  Using cached pandasai-2.3.2-py3-none-any.whl.metadata (10 kB)
  Using cached astor-0.8.1-py2.py3-none-any.whl.metadata (4.2 kB)
  Using cached duckdb-1.3.2-cp312-cp312-manylinux_2_27_x86_64.manylinux_2_28_x86_64.whl.metadata (7.0 kB)
  Using cached Faker-19.13.0-py3-none-any.whl.metadata (15 kB)
  Using cached openai-1.102.0-py3-none-any.whl.metadata (29 kB)
  Using cached pandas-1.5.3.tar.gz (5.2 MB)
  Installing build dependencies ... done
  Getting requirements to build wheel ... \anceled
^C
ERROR: Operation cancelled by user


In [ ]:
from pandasai import PandasAI
from pandasai.llm.openai import OpenAI
llm = OpenAI(api_token = "")

In [ ]:
pandas_ai = PandasAI(llm, conversational = False)

In [ ]:
df = pd.read_csv("/workspaces/ds-fall-2025-fri-0630/Week-01-Pandas/data/listings.csv")

In [ ]:
answer = pandas_ai.run(df, "What is the average price of listings in Brooklyn?")
print(answer)

### 2. (Very Easy) Find other cool Panda packages / add ons and show us what they can do well. And how you installed them. 

In [52]:
!pip install pandas_market_calendars


[notice] A new release of pip is available: 25.1.1 -> 25.2
[notice] To update, run: python3 -m pip install --upgrade pip


In [53]:
import pandas_market_calendars as mcal

In [54]:
nasdaq = mcal.get_calendar("NASDAQ")
schedule = nasdaq.schedule(start_date="2025-08-01", end_date="2025-08-31")
print(schedule.head())

                         market_open              market_close
2025-08-01 2025-08-01 13:30:00+00:00 2025-08-01 20:00:00+00:00
2025-08-04 2025-08-04 13:30:00+00:00 2025-08-04 20:00:00+00:00
2025-08-05 2025-08-05 13:30:00+00:00 2025-08-05 20:00:00+00:00
2025-08-06 2025-08-06 13:30:00+00:00 2025-08-06 20:00:00+00:00
2025-08-07 2025-08-07 13:30:00+00:00 2025-08-07 20:00:00+00:00


In [55]:
sessions = schedule.index
print(sessions)

DatetimeIndex(['2025-08-01', '2025-08-04', '2025-08-05', '2025-08-06',
               '2025-08-07', '2025-08-08', '2025-08-11', '2025-08-12',
               '2025-08-13', '2025-08-14', '2025-08-15', '2025-08-18',
               '2025-08-19', '2025-08-20', '2025-08-21', '2025-08-22',
               '2025-08-25', '2025-08-26', '2025-08-27', '2025-08-28',
               '2025-08-29'],
              dtype='datetime64[ns]', freq=None)


### 3. (Medium) Use the grammys.csv data for the next section of questions.

1. Who was won Album of the Year in 2016?


2. Who won Best Rap Album in 2009?


3. How many awards was Kendrick Lamar nomiated for, and how many did he win...?

In [56]:
df = pd.read_csv("/workspaces/ds-fall-2025-fri-0630/Week-01-Pandas/data/grammys.csv")
df.head()

,year,category,nominee,workers,winner
0,1959,Album of the Year,The Music from Peter Gunn.,Henry Mancini,True
1,1959,Best Album Cover,Only the Lonely,Frank Sinatra (art director),True
2,1959,Best Arrangement,The Music From Peter Gunn,Henry Mancini (artist/arranger),True
3,1959,Best Classical Performance - Chamber Music (in...,Beethoven: Quartet 130,"The Hollywood String Quartet, Paul Shure (arti...",True
4,1959,Best Classical Performance - Instrumentalist (...,Segovia Golden Jubilee,Andrés Segovia,True


In [57]:
album_winner_2016 = df[(df["year"] == 2016) & (df["category"] == "Album of the Year") & (df["winner"] == True)] 
print(album_winner_2016[["workers", "nominee"]])

                                                workers nominee
5505  Taylor Swift (artist/producer), Jack Antonoff ...   1989.


In [58]:
best_rap_2009 = df[(df["year"] == 2009) & (df["category"] == "Best Rap Album") & (df["winner"] == True)]
print(best_rap_2009[["workers", "nominee"]])

         workers         nominee
4396  Lil' Wayne  Tha Carter III


In [59]:
nominations_kl = df[df["nominee"].str.contains("Kendrick Lamar", na = False)]
nominations_kl_count = len(nominations_kl)
win_count = nominations_kl["winner"].sum()
print("Nominations: ", nominations_kl_count)
print("Wins: ", win_count)

Nominations:  2
Wins:  1


### (Hard) Load the Game Logs for 2022 and add the column names using a dictionary.  
* [Link to the data page](https://www.retrosheet.org/gamelogs/)
* [Link to the column names](https://procatinator.com/)
* haha, gotta find them yourself!

In [60]:
import pandas as pd

In [61]:
df_games = pd.read_csv("/workspaces/ds-fall-2025-fri-0630/Week-01-Pandas/data/gamelogs/gl2022.txt", header = None)
print(df_games.head())

        0    1    2    3   4    5    6   7    8    9    ...              151  \
0  20220407    0  Thu  SDN  NL    1  ARI  NL    1    2  ...        Seth Beer   
1  20220407    0  Thu  CIN  NL    1  ATL  NL    1    6  ...  Travis d'Arnaud   
2  20220407    0  Thu  MIL  NL    1  CHN  NL    1    4  ...    Jason Heyward   
3  20220407    0  Thu  PIT  NL    1  SLN  NL    1    0  ...    Yadier Molina   
4  20220407    0  Thu  NYN  NL    1  WAS  NL    1    5  ...    Maikel Franco   

   152       153              154  155       156              157  158  159  \
0   10  ellid002       Drew Ellis    5  perdg001  Gerardo Perdomo    6  NaN   
1    2  dicka001   Alex Dickerson   10  swand001   Dansby Swanson    6  NaN   
2    8  wisdp001   Patrick Wisdom    5  hoern001     Nico Hoerner    6  NaN   
3    2  badeh001   Harrison Bader    8  edmat001      Tommy Edman    4  NaN   
4    5  escoa003  Alcides Escobar    6  roblv001    Victor Robles    8  NaN   

  160  
0   Y  
1   Y  
2   Y  
3   Y  
4   

my source : https://www.retrosheet.org/gamelogs/glfields.txt

In [62]:
column_list = [
    "date", "number_of_game", "day_of_week", "visiting_team", "visiting_league",
    "visiting_game_number", "home_team", "home_league", "home_game_number",
    "visitor_score", "home_score", "length_of_game_outs", "day_night", "completion",
    "forfeit", "protest", "park_id", "attendance", "time_of_game_minutes",
    "visitor_line_score", "home_line_score", "visitor_ab", "visitor_hits",
    "visitor_doubles", "visitor_triples", "visitor_homeruns", "visitor_rbi",
    "visitor_sac_hits", "visitor_sac_flies", "visitor_hbp", "visitor_bb",
    "visitor_ibb", "visitor_k", "visitor_sb", "visitor_cs", "visitor_gdp",
    "visitor_ci", "visitor_lob", "visitor_pitchers", "visitor_er", "visitor_wp",
    "visitor_balks", "visitor_po", "visitor_a", "visitor_errors", "visitor_pb",
    "visitor_dp", "visitor_tp", "home_ab", "home_hits", "home_doubles",
    "home_triples", "home_homeruns", "home_rbi", "home_sac_hits", "home_sac_flies",
    "home_hbp", "home_bb", "home_ibb", "home_k", "home_sb", "home_cs", "home_gdp",
    "home_ci", "home_lob", "home_pitchers", "home_er", "home_wp", "home_balks",
    "home_po", "home_a", "home_errors", "home_pb", "home_dp", "home_tp",
    "ump_home_id", "ump_home_name", "ump_1b_id", "ump_1b_name", "ump_2b_id",
    "ump_2b_name", "ump_3b_id", "ump_3b_name", "ump_lf_id", "ump_lf_name",
    "ump_rf_id", "ump_rf_name", "visiting_manager_id", "visiting_manager_name",
    "home_manager_id", "home_manager_name", "winning_pitcher_id",
    "winning_pitcher_name", "losing_pitcher_id", "losing_pitcher_name",
    "saving_pitcher_id", "saving_pitcher_name", "game_winning_rbi_id",
    "game_winning_rbi_name", "visitor_starting_pitcher_id",
    "visitor_starting_pitcher_name", "home_starting_pitcher_id",
    "home_starting_pitcher_name", "visitor_player1_id", "visitor_player1_name",
    "visitor_player1_def_pos", "visitor_player2_id", "visitor_player2_name",
    "visitor_player2_def_pos", "visitor_player3_id", "visitor_player3_name",
    "visitor_player3_def_pos", "visitor_player4_id", "visitor_player4_name",
    "visitor_player4_def_pos", "visitor_player5_id", "visitor_player5_name",
    "visitor_player5_def_pos", "visitor_player6_id", "visitor_player6_name",
    "visitor_player6_def_pos", "visitor_player7_id", "visitor_player7_name",
    "visitor_player7_def_pos", "visitor_player8_id", "visitor_player8_name",
    "visitor_player8_def_pos", "visitor_player9_id", "visitor_player9_name",
    "visitor_player9_def_pos", "home_player1_id", "home_player1_name",
    "home_player1_def_pos", "home_player2_id", "home_player2_name",
    "home_player2_def_pos", "home_player3_id", "home_player3_name",
    "home_player3_def_pos", "home_player4_id", "home_player4_name",
    "home_player4_def_pos", "home_player5_id", "home_player5_name",
    "home_player5_def_pos", "home_player6_id", "home_player6_name",
    "home_player6_def_pos", "home_player7_id", "home_player7_name",
    "home_player7_def_pos", "home_player8_id", "home_player8_name",
    "home_player8_def_pos", "home_player9_id", "home_player9_name",
    "home_player9_def_pos", "additional_info", "acquisition_info"
]

In [63]:
col_dict = {i: name for i, name in enumerate(column_list)}
df_games = df_games.rename(columns=col_dict)

In [64]:
print(df_games.head())

       date  number_of_game day_of_week visiting_team visiting_league  \
0  20220407               0         Thu           SDN              NL   
1  20220407               0         Thu           CIN              NL   
2  20220407               0         Thu           MIL              NL   
3  20220407               0         Thu           PIT              NL   
4  20220407               0         Thu           NYN              NL   

   visiting_game_number home_team home_league  home_game_number  \
0                     1       ARI          NL                 1   
1                     1       ATL          NL                 1   
2                     1       CHN          NL                 1   
3                     1       SLN          NL                 1   
4                     1       WAS          NL                 1   

   visitor_score  ...  home_player8_id  home_player8_name  \
0              2  ...        Seth Beer                 10   
1              6  ...  Travis d'Arna

### (Extra Hard) Download the files for the past 5 years into a new folder and add them all into one data frame using pandas, then save that new file.
* Try to not hard code in the file names. We want to do this programmatically because what if we want to add new/more file names in the future.

In [65]:
import os

In [66]:
path = "/workspaces/ds-fall-2025-fri-0630/Week-01-Pandas/data/gamelogs/"

In [67]:
files = os.listdir(path)
print("Files found:", files)

Files found: ['gl2020.txt', 'gl2021.txt', 'gl2022.txt', 'gl2024.txt', 'gamelogs_2020_4.csv', 'gl2023.txt']


In [68]:
all_dfs = []
for file in files:
    if file.endswith(".txt"): 
        temp_df = pd.read_csv(path + file, header=None)
        all_dfs.append(temp_df)

In [69]:
df_all = pd.concat(all_dfs, ignore_index=True)

In [70]:
df_all.head()

,0,1,2,3,4,5,6,7,8,9,...,151,152,153,154,155,156,157,158,159,160
0,20200723,0,Thu,SFN,NL,1,LAN,NL,1,1,...,Joc Pederson,7,polla001,A.J. Pollock,10,barna001,Austin Barnes,2,NaN,Y
1,20200723,0,Thu,NYA,AL,1,WAS,NL,1,4,...,Asdrubal Cabrera,5,steva001,Andrew Stevenson,7,roblv001,Victor Robles,8,NaN,Y
2,20200724,0,Fri,MIL,NL,1,CHN,NL,1,0,...,Victor Caratini,10,hoern001,Nico Hoerner,4,happi001,Ian Happ,8,NaN,Y
3,20200724,0,Fri,DET,AL,1,CIN,NL,1,1,...,Nick Senzel,8,galvf001,Freddy Galvis,6,casac001,Curt Casali,2,NaN,Y
4,20200724,0,Fri,SFN,NL,2,LAN,NL,2,1,...,Chris Taylor,7,pedej001,Joc Pederson,10,smitw003,Will Smith,2,NaN,Y


In [71]:
col_dict = {i: name for i, name in enumerate(column_list)}
df_all = df_all.rename(columns=col_dict)

In [72]:
df_all.head()

,date,number_of_game,day_of_week,visiting_team,visiting_league,visiting_game_number,home_team,home_league,home_game_number,visitor_score,...,home_player8_id,home_player8_name,home_player8_def_pos,home_player9_id,home_player9_name,home_player9_def_pos,additional_info,acquisition_info,159,160
0,20200723,0,Thu,SFN,NL,1,LAN,NL,1,1,...,Joc Pederson,7,polla001,A.J. Pollock,10,barna001,Austin Barnes,2,NaN,Y
1,20200723,0,Thu,NYA,AL,1,WAS,NL,1,4,...,Asdrubal Cabrera,5,steva001,Andrew Stevenson,7,roblv001,Victor Robles,8,NaN,Y
2,20200724,0,Fri,MIL,NL,1,CHN,NL,1,0,...,Victor Caratini,10,hoern001,Nico Hoerner,4,happi001,Ian Happ,8,NaN,Y
3,20200724,0,Fri,DET,AL,1,CIN,NL,1,1,...,Nick Senzel,8,galvf001,Freddy Galvis,6,casac001,Curt Casali,2,NaN,Y
4,20200724,0,Fri,SFN,NL,2,LAN,NL,2,1,...,Chris Taylor,7,pedej001,Joc Pederson,10,smitw003,Will Smith,2,NaN,Y


In [73]:
df_all.to_csv(path + "gamelogs_2020_4.csv", index = False)